Analysis of TOPAS-nBio simulations of cells with nanoparticles

This notebook analyses the results of the simulations of TOPAS and TOPAS-nbio of cells with nanoparticles irradiated with I125 radiation source.

In [1]:
import sys
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint

# Import our custom modules
from dnadamage_phsp_manager import *
import sddparser
from chemistry_output_manager import *

In [ ]:
# Multirun processing
nruns=100 
runs_filebases = [f'/home/radiofisica/hector/mytopassimulations/TOPAS_CellsNPs/work/CellColony-med1-cell1/cell1/run{i+1}/' for i in range(nruns)]

Cell_results_files = {
    'Original_hists': 'DoseToCell_I125Beam.csv',
    'DoseToNucl_ph2': 'DoseNucleus_Total.csv',
    'DoseToNucl_ph3': 'DoseNucleus_Ph3.csv',
    'Ecell': 'EnergyToCell.csv',
    'NP_el': 'PhaseSpace_NP',
}

Cell_results = {
    'Original_hists': 0,
    'DoseToNucl_ph2': 0,
    'DoseToNucl_ph3': 0,
    'Ecell': 0,
    'NP_el': 0,
}

# Dictionary to keep track of total histories per result type
histories_with_scorer = {key: 0 for key in Cell_results.keys()}

# Function to process CSV files (dose, energy deposits)
def process_csv_file(file_path):
    """Process TOPAS CSV files and extract the sum value"""
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            
        # Find the data line (last non-comment line)
        data_line = None
        for line in lines:
            if not line.startswith('#'):
                data_line = line
                break
                
        if data_line:
            # Split by comma and get the sum (first value)
            values = data_line.strip().split(',')
            if len(values) >= 1:
                sum_value = float(values[0])
                # Get histories with scorer active (last value)
                histories = int(values[-1]) if len(values) >= 7 else 0
                return sum_value, histories
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
    
    return 0, 0

# Function to process Original_hists differently
def process_original_hists(file_path):
    """Extract the last value from the specified CSV file for Original_hists"""
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            
        # Find the data line (last non-comment line)
        data_line = None
        for line in lines:
            if not line.startswith('#'):
                data_line = line
                break
                
        if data_line:
            # Split by comma and get the last value
            values = data_line.strip().split(',')
            if len(values) >= 1:
                return int(values[-1])
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
    
    return 0

# Function to count phase space particles
def count_phsp_particles(file_base):
    """Count particles in phase space file"""
    try:
        header_file = file_base + ".header"
        with open(header_file, 'r') as f:
            lines = f.readlines()
            
        for line in lines:
            if "Number of Scored Particles:" in line:
                parts = line.strip().split(":")
                if len(parts) >= 2:
                    return int(parts[1].strip()), 0
    except Exception as e:
        print(f"Error processing phase space file {file_base}: {e}")
    
    return 0, 0

# Process all runs
for run_idx, run_base in enumerate(runs_filebases):
    print(f"Processing run {run_idx+1}/{nruns}...", end='\r')
    
    # Process each result type
    for result_key, file_name in Cell_results_files.items():
        file_path = os.path.join(run_base, file_name)
        if result_key == 'NP_el':
            # For phase space files
            count, _ = count_phsp_particles(file_path)
            Cell_results[result_key] += count
        elif result_key == 'Original_hists':
            # Process Original_hists differently
            Cell_results[result_key] += process_original_hists(file_path)
        else:
            # For other CSV files
            sum_value, histories = process_csv_file(file_path)
            Cell_results[result_key] += sum_value
            histories_with_scorer[result_key] += histories

print("\nProcessing complete!")

# Display results
print("\nResults Summary:")
print("-" * 50)
for key, value in Cell_results.items():
    if key == 'NP_el':
        print(f"{key}: {value} particles")
    elif key == 'Original_hists':
        print(f"{key}: {value} (total from {nruns} runs)")
    else:
        unit = "MeV" if key == 'Ecell' else "Gy"
        print(f"{key}: {value:.6e} {unit} (total from {nruns} runs)")

# Calculate averages per history where applicable
print("\nResults per history:")
print("-" * 50)
for key, value in Cell_results.items():
    if key != 'NP_el' and key != 'Original_hists' and histories_with_scorer[key] > 0:
        unit = "MeV/hist" if key == 'Ecell' else "Gy/hist"
        avg_per_history = value / histories_with_scorer[key]
        print(f"{key}: {avg_per_history:.6e} {unit}")

Error processing /home/radiofisica/hector/mytopassimulations/TOPAS_CellsNPs/work/CellColony-med1-cell1/cell1/run1_DoseToCell_I125Beam.csv: [Errno 2] No such file or directory: '/home/radiofisica/hector/mytopassimulations/TOPAS_CellsNPs/work/CellColony-med1-cell1/cell1/run1_DoseToCell_I125Beam.csv'
Error processing /home/radiofisica/hector/mytopassimulations/TOPAS_CellsNPs/work/CellColony-med1-cell1/cell1/run1_DoseNucleus_Total.csv: [Errno 2] No such file or directory: '/home/radiofisica/hector/mytopassimulations/TOPAS_CellsNPs/work/CellColony-med1-cell1/cell1/run1_DoseNucleus_Total.csv'
Error processing /home/radiofisica/hector/mytopassimulations/TOPAS_CellsNPs/work/CellColony-med1-cell1/cell1/run1_DoseNucleus_Ph3.csv: [Errno 2] No such file or directory: '/home/radiofisica/hector/mytopassimulations/TOPAS_CellsNPs/work/CellColony-med1-cell1/cell1/run1_DoseNucleus_Ph3.csv'
Error processing /home/radiofisica/hector/mytopassimulations/TOPAS_CellsNPs/work/CellColony-med1-cell1/cell1/run1_E